<a href="https://colab.research.google.com/github/finiteautomata/sentiment-elecciones/blob/master/notebooks/Targeted_Sentiment_Analysis_BERT_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture 
!pip install datasets transformers ipdb

In [ ]:
from datasets import load_dataset, concatenate_datasets

dataset = load_dataset("finiteautomata/tsa")

dataset = concatenate_datasets(dataset.values())

Downloading:   0%|          | 0.00/936 [00:00<?, ?B/s]

Using custom data configuration finiteautomata--tsa-48c5e891fefc8aa8


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/finiteautomata___parquet/finiteautomata--tsa-48c5e891fefc8aa8/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

def split_dataset(dataset, test_size=0.2, random_state=None):
    df = dataset.to_pandas()
    id_noticias = df.groupby("id_noticia").count()["label"]

    train_ids, test_ids = train_test_split(id_noticias, test_size=test_size, random_state=random_state)
    train_ids, dev_ids = train_test_split(train_ids, test_size=test_size, random_state=random_state)

    df_train = df[df["id_noticia"].isin(train_ids.index)]
    df_dev = df[df["id_noticia"].isin(dev_ids.index)]
    df_test = df[df["id_noticia"].isin(test_ids.index)]
    
    columns = ["titulo", "id_noticia", "target", "label"]
    features = dataset.features

    train_dataset = Dataset.from_pandas(df_train[columns], features=features)
    dev_dataset = Dataset.from_pandas(df_dev[columns], features=features)
    test_dataset = Dataset.from_pandas(df_test[columns], features=features)

    return DatasetDict(train=train_dataset, dev=dev_dataset, test=test_dataset)



In [ ]:
id2label = dataset.features["label"].names

label2id = { k:i for i, k in enumerate(id2label)}

label2id

{'NEG': 0, 'NEU': 1, 'POS': 2}

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = 'dccuchile/bert-base-spanish-wwm-uncased'


tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 128

Downloading:   0%|          | 0.00/310 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/650 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
def tokenize_with_target(example):
    return tokenizer(example["titulo"], example["target"], truncation=True)

def tokenize_without_target(example):
    return tokenizer(example["titulo"], truncation=True)

Perfecto, el segundo no tiene target

## Evaluación

In [ ]:
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

def compute_metrics(pred):
    """
    Compute metrics for Trainer
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    metrics = {}

    report = classification_report(labels, preds, output_dict=True)

    for i, cat in enumerate(id2label):
        rep = report[str(i)]
        metrics[cat.lower()+" F1"] = rep["f1-score"]
        metrics[cat.lower()+" Precision"] = rep["precision"]
        metrics[cat.lower()+" Recall"] = rep["recall"]

    metrics["macro_f1"] = report["macro avg"]["f1-score"]
    metrics["macro_precision"] = report["macro avg"]["precision"]
    metrics["macro_recall"] = report["macro avg"]["recall"]

    metrics["acc"] = report["accuracy"]

    return metrics

## Entrenamiento

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

def run_experiments(dataset_dict, model_name, batch_size=32, epochs=5, warmup=0.1):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, return_dict=True, num_labels=len(id2label)
    )
    total_steps = (epochs * len(dataset_dict["train"])) // batch_size
    warmup_steps = int(warmup * total_steps)


    training_args = TrainingArguments(
        output_dir='./results/',
        num_train_epochs=epochs,
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_steps=warmup_steps,
        do_eval=False,
        weight_decay=0.01,
        logging_dir='./logs',
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=lambda x: compute_metrics(x),
        train_dataset=dataset_dict["train"],
        data_collator=DataCollatorWithPadding(tokenizer),
        eval_dataset=dataset_dict["dev"],
    )

    trainer.train()

    return trainer.evaluate(dataset_dict["test"])

## 10-fold Monte Carlo Validation

In [ ]:
import time
import random
from tqdm.auto import tqdm
from transformers import set_seed

folds = 30
target_results = []
no_target_results = []


titulares = []

for i in tqdm(range(folds)):
    seed = time.time()
    set_seed(int(seed))
    test_size = random.normalvariate(0.2, 0.02)
    print(f"Epoch no {i+1}")
    print(f"{test_size:.3f} test size" )

    
    ds_dict = split_dataset(dataset, test_size=test_size, random_state=int(seed))

    
    targeted_dataset = ds_dict.map(tokenize_with_target)
    not_targeted_dataset = ds_dict.map(tokenize_without_target)

    titulares_test = sorted({ex["titulo"] for ex in ds_dict["test"]})
    titulares.append(titulares_test)

    target_results.append(run_experiments(targeted_dataset, model_name))
    no_target_results.append(run_experiments(not_targeted_dataset, model_name))


  0%|          | 0/30 [00:00<?, ?it/s]

Parameter 'function'=<function tokenize_with_target at 0x7f370a7fc320> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Epoch no 1
0.190 test size


  0%|          | 0/1595 [00:00<?, ?ex/s]

  0%|          | 0/382 [00:00<?, ?ex/s]

...

Epoch,Training Loss,Validation Loss,Neg f1,Neg precision,Neg recall,Neu f1,Neu precision,Neu recall,Pos f1,Pos precision,Pos recall,Macro F1,Macro Precision,Macro Recall,Acc
1,No log,0.894085,0.539823,0.549550,0.530435,0.406593,0.536232,0.327434,0.652632,0.579439,0.746988,0.533016,0.555074,0.534952,0.563452
2,No log,0.767383,0.683333,0.656000,0.713043,0.589212,0.554688,0.628319,0.710098,0.773050,0.656627,0.660881,0.661246,0.665996,0.664975
3,No log,0.792981,0.707317,0.664122,0.756522,0.589286,0.594595,0.584071,0.716981,0.750000,0.686747,0.671195,0.669572,0.675780,0.677665
4,No log,0.824752,0.709402,0.697479,0.721739,0.558140,0.588235,0.530973,0.731563,0.716763,0.746988,0.666368,0.667492,0.666567,0.677665
5,No log,0.865316,0.684444,0.700000,0.669565,0.557522,0.557522,0.557522,0.718101,0.707602,0.728916,0.653356,0.655041,0.652001,0.662437


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: titulo, id_noticia, target. If titulo, id_noticia, target are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 394
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-49
Configuration saved in ./results/checkpoint-49/config.json
Model weights saved in ./results/checkpoint-49/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: titulo, id_noticia, target. If titulo, id_noticia, target are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 394
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-98
Configuration saved in ./resul

loading configuration file https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/2416dab24674c27b5521594d6aa0929fc843a024c96711b1b5015cdff867291f.afa3630b664b4bd3e82d41660bdb96ec13236bbceadb0ae7c45c7c19f58652c7
Model config BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_toke

Epoch,Training Loss,Validation Loss,Neg f1,Neg precision,Neg recall,Neu f1,Neu precision,Neu recall,Pos f1,Pos precision,Pos recall,Macro F1,Macro Precision,Macro Recall,Acc
1,No log,0.892760,0.517413,0.604651,0.452174,0.494845,0.592593,0.424779,0.646310,0.559471,0.765060,0.552856,0.585572,0.547338,0.576142
2,No log,0.835443,0.619048,0.569343,0.678261,0.595918,0.553030,0.646018,0.652921,0.760000,0.572289,0.622629,0.627458,0.632189,0.624365
3,No log,0.835174,0.609442,0.601695,0.617391,0.603306,0.565891,0.646018,0.658147,0.700680,0.620482,0.623632,0.622756,0.627964,0.626904
4,No log,0.870286,0.620690,0.615385,0.626087,0.563107,0.623656,0.513274,0.691429,0.657609,0.728916,0.625075,0.632216,0.622759,0.637056
5,No log,0.909645,0.618644,0.603306,0.634783,0.604444,0.607143,0.601770,0.703364,0.714286,0.692771,0.642151,0.641578,0.643108,0.649746


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: titulo, id_noticia, target. If titulo, id_noticia, target are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 394
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-49
Configuration saved in ./results/checkpoint-49/config.json
Model weights saved in ./results/checkpoint-49/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: titulo, id_noticia, target. If titulo, id_noticia, target are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 394
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-98
Configuration saved in ./resul

In [25]:
import numpy as np
import scipy.stats
import pandas as pd

try:
    data = [
        *[{**{"target": "targeted"}, **res} for res in target_results],
        *[{**{"target": "none"}, **res} for res in no_target_results]
    ]

    df_res = pd.DataFrame(data)
    df_res.to_csv("/content/drive/MyDrive/sentiment-targeted-results.csv")
except:
    df_res = pd.read_csv("/content/drive/MyDrive/sentiment-targeted-results.csv")
    target_results = df_res[df_res["target"] == "targeted"].to_dict('records')
    no_target_results = df_res[df_res["target"] == "none"].to_dict('records')

In [26]:



df_mean = (df_res.groupby("target").mean() * 100).round(2)
df_std = (df_res.groupby("target").std() * 100).round(1)

df_str_res = df_mean.astype(str) + " +- " + df_std.astype(str)

df_str_res

,Unnamed: 0,eval_loss,eval_neg F1,eval_neg Precision,eval_neg Recall,eval_neu F1,eval_neu Precision,eval_neu Recall,eval_pos F1,eval_pos Precision,eval_pos Recall,eval_macro_f1,eval_macro_precision,eval_macro_recall,eval_acc,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
target,,,,,,,,,,,,,,,,,,,
none,4450.0 +- 880.3,91.07 +- 6.4,61.7 +- 2.8,60.38 +- 3.5,63.69 +- 6.5,58.81 +- 4.0,62.62 +- 3.9,55.64 +- 5.3,67.09 +- 2.3,65.73 +- 3.2,68.78 +- 4.4,62.53 +- 2.0,62.91 +- 1.9,62.71 +- 2.1,63.13 +- 1.9,71.83 +- 5.8,70065.38 +- 1908.8,2246.61 +- 76.5,500.0 +- 0.0
targeted,1450.0 +- 880.3,85.04 +- 5.6,69.37 +- 2.6,69.3 +- 4.4,69.77 +- 3.9,60.02 +- 3.6,63.6 +- 4.5,57.0 +- 4.1,71.69 +- 2.2,69.2 +- 2.6,74.49 +- 3.3,67.03 +- 1.8,67.36 +- 1.8,67.09 +- 1.8,67.58 +- 1.8,77.38 +- 6.9,65095.88 +- 2265.6,2086.88 +- 73.5,500.0 +- 0.0


In [27]:
df_str_res.columns

Index(['Unnamed: 0', 'eval_loss', 'eval_neg F1', 'eval_neg Precision',
       'eval_neg Recall', 'eval_neu F1', 'eval_neu Precision',
       'eval_neu Recall', 'eval_pos F1', 'eval_pos Precision',
       'eval_pos Recall', 'eval_macro_f1', 'eval_macro_precision',
       'eval_macro_recall', 'eval_acc', 'eval_runtime',
       'eval_samples_per_second', 'eval_steps_per_second', 'epoch'],
      dtype='object')

In [28]:
columns = [
            "eval_neg Precision", "eval_neg Recall", "eval_neg F1", 
            "eval_neu Precision", "eval_neu Recall", "eval_neu F1", 
            "eval_pos Precision", "eval_pos Recall", "eval_pos F1", 
            "eval_macro_precision", "eval_macro_recall", "eval_macro_f1", 
]
df_str_res[columns].T

target,none,targeted
eval_neg Precision,60.38 +- 3.5,69.3 +- 4.4
eval_neg Recall,63.69 +- 6.5,69.77 +- 3.9
eval_neg F1,61.7 +- 2.8,69.37 +- 2.6
eval_neu Precision,62.62 +- 3.9,63.6 +- 4.5
eval_neu Recall,55.64 +- 5.3,57.0 +- 4.1
eval_neu F1,58.81 +- 4.0,60.02 +- 3.6
eval_pos Precision,65.73 +- 3.2,69.2 +- 2.6
eval_pos Recall,68.78 +- 4.4,74.49 +- 3.3
eval_pos F1,67.09 +- 2.3,71.69 +- 2.2
eval_macro_precision,62.91 +- 1.9,67.36 +- 1.8


In [29]:
print(df_str_res[columns].T.to_latex())

\begin{tabular}{lll}
\toprule
target &          none &      targeted \\
\midrule
eval\_neg Precision   &  60.38 +- 3.5 &   69.3 +- 4.4 \\
eval\_neg Recall      &  63.69 +- 6.5 &  69.77 +- 3.9 \\
eval\_neg F1          &   61.7 +- 2.8 &  69.37 +- 2.6 \\
eval\_neu Precision   &  62.62 +- 3.9 &   63.6 +- 4.5 \\
eval\_neu Recall      &  55.64 +- 5.3 &   57.0 +- 4.1 \\
eval\_neu F1          &  58.81 +- 4.0 &  60.02 +- 3.6 \\
eval\_pos Precision   &  65.73 +- 3.2 &   69.2 +- 2.6 \\
eval\_pos Recall      &  68.78 +- 4.4 &  74.49 +- 3.3 \\
eval\_pos F1          &  67.09 +- 2.3 &  71.69 +- 2.2 \\
eval\_macro\_precision &  62.91 +- 1.9 &  67.36 +- 1.8 \\
eval\_macro\_recall    &  62.71 +- 2.1 &  67.09 +- 1.8 \\
eval\_macro\_f1        &  62.53 +- 2.0 &  67.03 +- 1.8 \\
\bottomrule
\end{tabular}



In [30]:
target_results[0]

{'Unnamed: 0': 0,
 'epoch': 5.0,
 'eval_acc': 0.6948051948051948,
 'eval_loss': 0.8235428929328918,
 'eval_macro_f1': 0.6895297773162327,
 'eval_macro_precision': 0.6919751405600802,
 'eval_macro_recall': 0.6939479106419896,
 'eval_neg F1': 0.7054545454545454,
 'eval_neg Precision': 0.6598639455782312,
 'eval_neg Recall': 0.7578125,
 'eval_neu F1': 0.6231884057971016,
 'eval_neu Precision': 0.6935483870967742,
 'eval_neu Recall': 0.5657894736842105,
 'eval_pos F1': 0.7399463806970509,
 'eval_pos Precision': 0.7225130890052356,
 'eval_pos Recall': 0.7582417582417582,
 'eval_runtime': 0.7057,
 'eval_samples_per_second': 654.642,
 'eval_steps_per_second': 21.255,
 'target': 'targeted'}

In [32]:
pvals = []
for col in columns:
    print(col)
    pval = scipy.stats.wilcoxon(
        x=[r[col] for r in target_results],
        y=[r[col] for r in no_target_results],
        alternative="greater"
    )
    print(pval)
    pvals.append((col, pval))

eval_neg Precision
WilcoxonResult(statistic=465.0, pvalue=8.671988141602892e-07)
eval_neg Recall
WilcoxonResult(statistic=385.0, pvalue=1.7035299488990043e-05)
eval_neg F1
WilcoxonResult(statistic=465.0, pvalue=8.671988141602892e-07)
eval_neu Precision
WilcoxonResult(statistic=305.0, pvalue=0.06795389274820583)
eval_neu Recall
WilcoxonResult(statistic=322.0, pvalue=0.032817004382478435)
eval_neu F1
WilcoxonResult(statistic=327.0, pvalue=0.025965332514848578)
eval_pos Precision
WilcoxonResult(statistic=449.0, pvalue=4.233040845191937e-06)
eval_pos Recall
WilcoxonResult(statistic=456.5, pvalue=2.0383834447294846e-06)
eval_pos F1
WilcoxonResult(statistic=465.0, pvalue=8.671988141602892e-07)
eval_macro_precision
WilcoxonResult(statistic=464.0, pvalue=9.604605524515698e-07)
eval_macro_recall
WilcoxonResult(statistic=463.0, pvalue=1.0633180053657127e-06)
eval_macro_f1
WilcoxonResult(statistic=463.0, pvalue=1.0633180053657127e-06)


In [34]:
%%capture
!pip install statsmodels

In [41]:
pvals[0]

('eval_neg Precision',
 WilcoxonResult(statistic=465.0, pvalue=8.671988141602892e-07))

In [47]:
from statsmodels.stats import multitest

list(zip(
    columns,
    multitest.multipletests([x[1].pvalue for x in pvals], method="fdr_bh")[1]
))


[('eval_neg Precision', 2.1266360107314253e-06),
 ('eval_neg Recall', 2.2713732651986725e-05),
 ('eval_neg F1', 2.1266360107314253e-06),
 ('eval_neu Precision', 0.06795389274820583),
 ('eval_neu Recall', 0.0358003684172492),
 ('eval_neu F1', 0.031158399017818292),
 ('eval_pos Precision', 6.349561267787905e-06),
 ('eval_pos Recall', 3.494371619536259e-06),
 ('eval_pos F1', 2.1266360107314253e-06),
 ('eval_macro_precision', 2.1266360107314253e-06),
 ('eval_macro_recall', 2.1266360107314253e-06),
 ('eval_macro_f1', 2.1266360107314253e-06)]